In [17]:
import datetime
import sys
import string
import torch
sys.path.append("../")
import rdf_classify.extract as extract
import rdf_classify.load as load

In [4]:
dev_graphs = rdf_classify.extract.from_zipfile("../data/input/sinopia_export_dev_all_2020-02-02T00_01_16.216Z.zip")

Failed to parse sinopia_export_all_2020-02-02T00:01:16.216Z/tamu_2020-02-02T00:01:20.532Z/complete.log
Failed to parse sinopia_export_all_2020-02-02T00:01:16.216Z/washington_2020-02-02T00:01:18.182Z/complete.log
Failed to parse sinopia_export_all_2020-02-02T00:01:16.216Z/hrc_2020-02-02T00:01:16.408Z/complete.log
Failed to parse sinopia_export_all_2020-02-02T00:01:16.216Z/alberta_2020-02-02T00:01:18.044Z/complete.log
Failed to parse sinopia_export_all_2020-02-02T00:01:16.216Z/ucsd_2020-02-02T00:01:19.861Z/complete.log
Failed to parse sinopia_export_all_2020-02-02T00:01:16.216Z/boulder_2020-02-02T00:01:17.149Z/complete.log
Failed to parse sinopia_export_all_2020-02-02T00:01:16.216Z/minnesota_2020-02-02T00:01:17.268Z/complete.log
Failed to parse sinopia_export_all_2020-02-02T00:01:16.216Z/pcc_2020-02-02T00:01:16.575Z/complete.log
Failed to parse sinopia_export_all_2020-02-02T00:01:16.216Z/princeton_2020-02-02T00:01:20.598Z/complete.log
Failed to parse sinopia_export_all_2020-02-02T00:01:1

In [5]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [7]:
resource_templates = {}
for graph in dev_graphs:
    rt_query = graph.query("""SELECT ?subj ?resource_template WHERE {
 ?subj <http://sinopia.io/vocabulary/hasResourceTemplate> ?resource_template
}""")
    for row in rt_query:
        subject, rt_key = str(row[0]), str(row[1]) # Ugly
        if not rt_key in resource_templates:
            resource_templates[rt_key] = {}
        for predicate in graph.predicates(subject=row[0]):
            pred_key = str(predicate)
            if pred_key in resource_templates[rt_key]:
                resource_templates[rt_key][pred_key] += 1
            else:
                resource_templates[rt_key][pred_key] = 1

In [22]:
VALID_URL = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-._~:/?#[]@!$&'()*+,;="

In [30]:
def uri_to_matrix(uri):
    matrix = []
    for char in uri:
        position = VALID_URL.find(char)
        if position < 0:
            continue # Ignore for now
        for i in range(0, len(VALID_URL)):
            if i == position:
                vector.append(1.0)
            else:
                vector.append(0.0)
        
        matrix.append(vector)
    return matrix

In [32]:
uri_matrix = uri_to_matrix('http://example.com/')

In [34]:
uri_matrix[1]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [10]:
def letterToIndex(letter):
    return all_letters.find(letter)

In [11]:
def uri_to_tensor(uri):
    tensor = torch.zeros(len(uri), 1, len(valid_url))
    for i, char in enumerate(uri):
        value_offset = valid_url.find(char)
        if value_offset < 0:
            raise ValueError(f"{char} not found in valid URL characters")
        tensor[i][0][value_offset] = 1
    return tensor
        

In [13]:
uri_to_tensor("http://example.com/").size()

torch.Size([19, 1, 84])

In [ ]:
import torch.nn as nn

